In [3]:
import nltk
import pandas as pd
from corextopic import corextopic as ct

In [2]:
!pip install corextopic

In [22]:
df=pd.read_csv("C:\\Users\\lenovo\\Desktop\\IR\\Project\\mask-filtered-preprocessed.csv")

In [23]:
df.head()

,Unnamed: 0,lemmas_back_to_text,lemma_tokens
0,0,read coronavirus say face mask useless good wa...,"['read', 'coronavirus', 'say', 'face', 'mask',..."
1,1,let face democrats coronavirus destroy economy...,"['let', 'face', 'democrats', 'coronavirus', 'd..."
2,2,patient ask wear mask work wear afraid corona ...,"['patient', 'ask', 'wear', 'mask', 'work', 'we..."
3,3,start cry way home wipe face coronavirus,"['start', 'cry', 'way', 'home', 'wipe', 'face'..."
4,4,corona virus panic realize touch face day,"['corona', 'virus', 'panic', 'realize', 'touch..."


In [24]:
df=df.drop(columns=["Unnamed: 0"])

In [25]:
df.lemmas_back_to_text=df.lemmas_back_to_text.astype("str")

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33074 entries, 0 to 33073
Data columns (total 2 columns):
lemmas_back_to_text    33074 non-null object
lemma_tokens           33074 non-null object
dtypes: object(2)
memory usage: 516.9+ KB


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df['lemmas_back_to_text'])
tfidf = vectorizer.transform(df['lemmas_back_to_text'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

4525


In [32]:
vocab

['00',
 '000',
 '000 american',
 '000 coronavirus',
 '000 dead',
 '000 death',
 '000 face',
 '000 life',
 '000 new',
 '000 people',
 '10',
 '10 day',
 '100',
 '100 000',
 '100 day',
 '100 face',
 '1000',
 '104',
 '104 98',
 '11',
 '12',
 '13',
 '14',
 '14 day',
 '15',
 '150',
 '16',
 '17',
 '18',
 '19',
 '19 affect',
 '19 airborne',
 '19 america',
 '19 case',
 '19 come',
 '19 community',
 '19 continue',
 '19 control',
 '19 coronavirus',
 '19 crisis',
 '19 datum',
 '19 death',
 '19 exist',
 '19 face',
 '19 force',
 '19 garment',
 '19 go',
 '19 hoax',
 '19 infection',
 '19 joke',
 '19 kill',
 '19 mask',
 '19 mutation',
 '19 need',
 '19 number',
 '19 outbreak',
 '19 pandemic',
 '19 patient',
 '19 people',
 '19 positive',
 '19 protection',
 '19 real',
 '19 recovery',
 '19 recycle',
 '19 relief',
 '19 restriction',
 '19 safety',
 '19 seriously',
 '19 spread',
 '19 stay',
 '19 surge',
 '19 temp',
 '19 test',
 '19 testing',
 '19 transmission',
 '19 update',
 '19 vaccine',
 '19 victim',
 '19 v

In [46]:
anchors = []
model = ct.Corex(n_hidden=10, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [47]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: bottle face, soda bottle, soda, recycle soda, recycle, bottle, shield hero, hero, coronavirus recycle, face shield
Topic #2: covid, 19, covid 19, mandate, mask mandate, mask covid, case, statewide, statewide mask, 19 case
Topic #3: wear, wear mask, mask wear, wear damn, people wear, refuse wear, damn mask, damn, wear fucking, refuse
Topic #4: stay, distance, hand, wash, social, wash hand, stay home, home, social distance, mask social
Topic #5: distancing, social distancing, practice, practice social, mask practice, distancing wear, distancing mask, short live, abandoned mask, abandoned
Topic #6: face, face mask, wear face, face cover, cover, mask face, cloth face, cloth, face coronavirus, cover face
Topic #7: public, spread, mask public, covering, face covering, require, help, prevent, mask help, wearamask
Topic #8: order, county, mandatory, store, mask order, new, requirement, enforce, rule, mask requirement
Topic #9: people, trump, save, mask people, die, life, house, save 

In [37]:
anchor=[
    ["soda","soda bottle face","soda bottle"],
    ["covid","19","covid 19"],
    ["face mask","mask","wear","mandate","face cover","covering","wear face","mask protect",'refuse wear'],
    ["social","distancing","distance"],
    ["stay home","home",],
    ['public'],
    ['damn','fucking'],
    ['people','face']
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=10, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [38]:

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: covid, 19, covid 19, mandate, mask mandate, mask covid, statewide, case, statewide mask, 19 case
Topic #2: social, distance, hand, wash, wash hand, distancing, spread, social distancing, social distance, mask social
Topic #3: damn, damn mask, wear damn, refuse, fucking, refuse wear, protect, mask protect, fucking mask, wear fucking
Topic #4: soda bottle, bottle face, soda, recycle soda, bottle, recycle, shield hero, hero, coronavirus recycle, face shield
Topic #5: public, cover, require, mask public, face cover, covering, face covering, wear face, store, use
Topic #6: wear, wear mask, face, face mask, mask wear, need wear, covid19 wear, wo wear, anti mask, face face
Topic #7: people, people wear, die, mask people, get, day, slap, slap face, year, let
Topic #8: stay, home, stay home, safe, mask stay, save, life, stay safe, home wear, save life


In [39]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(8)]
).astype(float)
topic_df.index = df.index
df = pd.concat([df, topic_df], axis=1)


In [41]:
df.sample(10, random_state=42)

,lemmas_back_to_text,lemma_tokens,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8
2647,forage supply dress plague doctor plague dr ma...,"['forage', 'supply', 'dress', 'plague', 'docto...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
30616,@drericding @hughes_eilir ventilation open win...,"['drericding', 'hughes_eilir', 'ventilation', ...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
14793,wanna know makeup rub inside mask dye hair lol,"['wanna', 'know', 'makeup', 'rub', 'inside', '...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12774,richland level 2 wear mask circle contact smal...,"['richland', 'level', '2', 'wear', 'mask', 'ci...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12678,wear fucking mask,"['wear', 'fucking', 'mask']",0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
24839,@yamiche consider chair @lindseygrahamsc tell ...,"['yamiche', 'consider', 'chair', 'lindseygraha...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4884,buy face mask glad set week rest order way thing,"['buy', 'face', 'mask', 'glad', 'set', 'week',...",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1588,7 week track covid19 1 872 case sf 500k case 2...,"['7', 'week', 'track', 'covid19', '1', '872', ...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5218,coronavirus recycle soda bottle face shield hero,"['coronavirus', 'recycle', 'soda', 'bottle', '...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11091,phase 3 way july 1 let mindful movement wear m...,"['phase', '3', 'way', 'july', '1', 'let', 'min...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [45]:
df[df.topic_1 == 1]

,lemmas_back_to_text,lemma_tokens,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8
5,@thesolarspirit covid-19 peak china rok china ...,"['thesolarspirit', 'covid-19', 'peak', 'china'...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,petition protect unt community covid-19 suspen...,"['petition', 'protect', 'unt', 'community', 'c...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,intentional murder careless covid-19 spreader ...,"['intentional', 'murder', 'careless', 'covid-1...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,fact 7 10 american covid-19 1st wave pandemic ...,"['fact', '7', '10', 'american', 'covid-19', '1...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
19,educate educate coronavirus covid-19 scary cov...,"['educate', 'educate', 'coronavirus', 'covid-1...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
41,take face value president announcement online ...,"['take', 'face', 'value', 'president', 'announ...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48,sense covid-19 discuss friday 03/13/2020 wear ...,"['sense', 'covid-19', 'discuss', 'friday', '03...",1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
51,covid-19 tip weekend feel touching face touch ...,"['covid-19', 'tip', 'weekend', 'feel', 'touchi...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
75,amidst uncertainty face covid-19 pandemic casa...,"['amidst', 'uncertainty', 'face', 'covid-19', ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,city council get coronavirus update councilman...,"['city', 'council', 'get', 'coronavirus', 'upd...",1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
